In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
import matplotlib.pyplot as plt
#%maplotlib inline
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import shap
import eval

isinstance('22',str)

In [2]:
def gen_data(objective,labels=None):
    from tqdm import tqdm
    if objective == 'binary':
        labels = [0,1]
        y_true = np.random.choice(labels, 10000, p=[0.7,0.3])
        y_score = np.random.normal(0,1,size=10000)
        y_score[y_true==1] = y_score[y_true==1]+0.5
        y_score[y_true==0] = y_score[y_true==0]-0.5
        y_score = 1/(1+np.exp(-1*y_score))
        y_pred = (y_score>=0.5).astype(np.int)
        y_score = np.c_[1-y_score,y_score]
        return y_true,y_pred,y_score
    elif objective == 'multi':
        if labels is  None:
            labels = [1,2,3,4,5]
        n = len(labels)
        p = np.random.rand(n)
        p = p/p.sum()
        y_true = np.random.choice(labels, 10000, p=p)
        y_score = pd.DataFrame(np.random.rand(10000,5))
        y_score = np.array(y_score.div(y_score.sum(axis=1),axis=0))
        y_pred = np.argmax(y_score,axis=1)
        y_pred = np.array(pd.Series(y_pred).replace(dict(zip(range(n),labels))))
        return y_true,y_pred,y_score
    elif objective == 'rank':
        '''
        假定有1000个用户，每个用户可推荐10个产品
        '''
        n = 10000
        data = pd.DataFrame(index=range(n),columns=['groupid','y_true','y_pred','y_score'])
        for u in tqdm(range(int(n/10))):
            index = np.arange(u*10,(u+1)*10)
            groupid = u+1
            y_true = np.zeros(10)
            t = np.random.choice([0,1,2],1,p=[0.05,0.60,0.35])[0]
            w = np.arange(0,10)
            np.random.shuffle(w)
            y_true[w[:t].tolist()] = 1
            w = np.arange(1,11)
            np.random.shuffle(w)
            y_pred = w
            w = dict(zip(np.arange(1,11),sorted(np.random.rand(10),reverse=True)))
            y_score = [w[i] for i in y_pred]
            data.loc[np.arange(u*10,(u+1)*10),'groupid'] = u+1
            data.loc[np.arange(u*10,(u+1)*10),'y_true'] = y_true
            data.loc[np.arange(u*10,(u+1)*10),'y_pred'] = y_pred
            data.loc[np.arange(u*10,(u+1)*10),'y_score'] = y_score
        data['y_true'] = data['y_true'].astype(np.int)
        data['y_pred'] = data['y_pred'].astype(np.int)
        data['y_score'] = data['y_score'].astype(np.double)
        return data['y_true'],data['y_pred'],data['y_score'],data['groupid']
    else:
        return None
        

In [3]:
y_true,y_pred,y_score = gen_data('binary')
result,crosstab = eval.score(y_true,y_pred,y_score,ordered=True)
display(result)
display(crosstab)

,precision,recall,auc,f1,acc
0,0.843097,0.691959,0.75856,0.760087,0.692
1,0.484453,0.692099,0.75856,0.569953,0.692


pred,0,1
true,,
0,4879,2172
1,908,2041


In [4]:
y_true,y_pred,y_score = gen_data('multi')
result,crosstab = eval.score(y_true,y_pred,y_score,ordered=True)
display(result)
display(crosstab)

,precision,recall,auc,f1,f1_micro,f1_macro,f1_weighted,acc,acc_lr
1,0.157683,0.189927,0.489634,0.172309,0.1977,0.196083,0.199319,0.1977,0.5327
2,0.223267,0.197634,0.500519,0.209670,0.1977,0.196083,0.199319,0.1977,0.5327
3,0.255416,0.201111,0.504573,0.225033,0.1977,0.196083,0.199319,0.1977,0.5327
4,0.185334,0.196800,0.498219,0.190895,0.1977,0.196083,0.199319,0.1977,0.5327
5,0.166914,0.201314,0.499215,0.182507,0.1977,0.196083,0.199319,0.1977,0.5327


pred,1,2,3,4,5
true,,,,,
1,313,346,313,337,339
2,444,451,441,464,482
3,509,528,507,481,496
4,383,353,405,369,365
5,336,342,319,340,337


In [5]:
y_true,y_pred,y_score,groupid = gen_data('rank')
result,crosstab = eval.score(y_true,y_pred,y_score,groupid=groupid)
display(result)
display(crosstab)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 108.05it/s]


,precision@k,recall@k,ndcg@k,support
1,0.107,0.107,0.107,107
2,0.122,0.244,0.191961,145
3,0.113333,0.34,0.237806,112
4,0.113,0.452,0.282444,137
5,0.1084,0.542,0.314119,115
6,0.106,0.636,0.342673,133
7,0.101286,0.709,0.362668,110
8,0.099125,0.793,0.381877,145
9,0.0971111,0.874,0.40147,139
10,0.0944,0.944,0.416292,131


y_pred,1,2,3,4,5,6,7,8,9,10
y_true,,,,,,,,,,
0,893,855,888,863,885,867,890,855,861,869
1,107,145,112,137,115,133,110,145,139,131


In [ ]:

def clf_report(y,yhat,yhat_proba,threshold_selection=False,class_label=None,acc_lr=False):
    '''分类模型评估
    :parameter
    y: samplesize，真实值
    yhat:samplesize，预测值
    yhat_proba:samplesize * n_classes,预测概率
    threshold_selection：是否计算各阈值下的p和r
    class_label:各类别的标签，dict格式，如{0:'负样本',1:'正样本'}
    '''

    classes=sorted(pd.unique(y))
    # 判断是否是二分类问题
    if classes == [0,1]:
        isbin = 1
    else:
        isbin = 0
        
    if class_label is None:
        classes_name=dict(zip(classes,['{}'.format(i) for i in classes]))
    else:
        classes_name=class_label

    confusion_matrix=pd.crosstab(y, yhat, rownames=['True'], colnames= ['Predicted'], margins=False)
    confusion_matrix=confusion_matrix.rename(index=classes_name,columns=classes_name)
    result=pr_score(y,yhat,yhat_proba,acc_lr)
    result=result.rename(columns=classes_name,index=classes_name)
    
    
#     if isbin == 1:
#         fig1,ax1=plt.subplots(1,1,figsize=(9*0.5,0.5*6*1))
#         classes = [1]
#         classes_name = {1:classes_name[1]}
#     else:
#         fig1,ax1=plt.subplots(len(classes),1,figsize=(9*0.5,0.5*6*len(classes)))

    fig1,ax1=plt.subplots(len(classes),1,figsize=(9*0.5,0.5*6*len(classes)))
    for ci,c in enumerate(classes):
        label1='{}'.format(classes_name[c])
        label2='+'.join([classes_name[i] for i in classes if i!=c])
        auc=result.loc[label1,'auc']
        p=result.loc[label1,'p']
        r=result.loc[label1,'r']
        sns.kdeplot(yhat_proba[y==c,ci],ax=ax1[ci],shade=True,label=label1)
        sns.kdeplot(yhat_proba[y!=c,ci],ax=ax1[ci],shade=True,label=label2)
        ax1[ci].autoscale()
        ax1[ci].set_xlabel('{} (auc={:.2f},p={:.2f},r={:.2f})'.format(label1,auc,p,r))

    ax1[0].set_title('Density Curve')
    fig1.tight_layout()
    fig1.show()
    
    
    output={'result':result,'confusion_matrix':confusion_matrix,'fig1':fig1}
    
    if threshold_selection:
        fig2,ax2=plt.subplots(len(classes),1,figsize=(9*0.5,0.5*6*len(classes)))
        for ci,c in enumerate(classes):
            label1='{}'.format(classes_name[c])
            label2='+'.join([classes_name[i] for i in classes if i!=c])
            auc=result.loc[label1,'auc']
            p=result.loc[label1,'p']
            r=result.loc[label1,'r']
            ty=np.array(y==c)
            tyhat_proba=yhat_proba[:,ci]
            threshold_list=np.arange(0,1,0.01)[1:]
            p_list=[]
            r_list=[]
            for t in threshold_list:
                tyhat=np.array(tyhat_proba>=t)
                p=metrics.precision_score(ty,tyhat,pos_label=1)
                if not any(tyhat):
                    p=1
                r=metrics.recall_score(ty,tyhat,pos_label=1)
                p_list+=[p]
                r_list+=[r]

            tn=np.where(np.array(p_list)>=np.array(r_list))[0][0]
            t_equal=threshold_list[tn]

            ax2[ci].plot(threshold_list,p_list,label='p')
            ax2[ci].plot(threshold_list,r_list,label='r')
            ax2[ci].axvline(x=t_equal,linestyle=':')
            if t_equal>=0.6:
                ax2[ci].text(t_equal,0.5,r't={:.2f},p={:.2f},r={:.2f}   '.format(t_equal,p_list[tn],r_list[tn]),horizontalalignment='right')
            else :
                ax2[ci].text(t_equal,0.5,r'   t={:.2f},p={:.2f},r={:.2f}'.format(t_equal,p_list[tn],r_list[tn]))
            ax2[ci].legend()
            ax2[ci].set_xlabel('class: {}'.format(label1))
            ax2[ci].set_xlim(0,1)
            ax2[ci].set_ylim(0,1)

        ax2[0].set_title('Threshold Selection')
        fig2.tight_layout()
        fig2.show()
        output['fig2']=fig2
        #fig2.savefig('Threshold Selection.png',dpi=400)

    print('\n =======混淆矩阵=========')
    display(confusion_matrix)
    print('=======模型效果=========')
    display(result)
    


    return output